In [3]:
import trimesh
import scipy
import numpy as np
import matplotlib.cm as cm
import nibabel as nib
import gdist
import sys
import pandas as pd
import importlib
from scipy import stats
import os
import json

In [38]:
f = open("/Users/dvarelat/Documents/MASTER/TFM/methods/specimens.json")
data = json.load(f)


[element for sublist in [[f"2019{e}" for e in data[i]] for i in ["stage6"]] for element in sublist]


['20190308_E2',
 '20190401_E1',
 '20190401_E2',
 '20190502_E1',
 '20190517_E2',
 '20190806_E1',
 '20190806_E2']

In [24]:
sys.path.insert(1, "/Users/dvarelat/Documents/MASTER/TFM/methods")
import cardiac_region
importlib.reload(cardiac_region)
import cardiac_region as c

sys.path.insert(1, "/Users/dvarelat/Documents/MASTER/TFM/surface_codes")
import array2PLY

In [27]:
ESPECIMEN = "20190516_E3"
features_all = "/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/features"

file_mesh = f"/Users/dvarelat/Documents/MASTER/TFM/midS/{ESPECIMEN}_myo_midS.ply"
line_mesh = f"/Users/dvarelat/Documents/MASTER/TFM/lines_ply_myo/line_{ESPECIMEN}_myo_10000.ply"
folder_surface = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/features/surfaces/myo"
image_path = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/RESULTS/membranes/GASP_PNAS/{ESPECIMEN}_mGFP_XYZ_predictions_GASP.nii.gz"
DFFILE = os.path.join(features_all, f"{ESPECIMEN}_cell_properties_radiomics.csv")



In [10]:
df_clean = pd.read_csv(DFFILE)
labels = list(df_clean.original_labels)
print(df_clean.shape)

(7865, 26)


In [11]:
d = 50
final_number_faces = 18000

In [12]:
proxy = nib.load(image_path)
img = np.asarray(proxy.get_fdata(), dtype=proxy.get_data_dtype())
print(img.shape)

(1024, 1024, 594)


In [28]:
dim_info = c.load3D_metadata(image_path)
mesh = trimesh.load_mesh(line_mesh, process=False)
mesh = mesh.simplify_quadratic_decimation(int(final_number_faces))
print(f"Numero de vertices {len(mesh.vertices)}")

Numero de vertices 5002


In [29]:
matrix = gdist.local_gdist_matrix(mesh.vertices, mesh.faces.astype(np.int32), d)
matrix_ij = scipy.sparse.find(matrix)
matrix_i, matrix_j, distances = matrix_ij[1], matrix_ij[0], matrix_ij[2]
j_coord = np.asarray(mesh.vertices[matrix_j])
len(j_coord)

683612

In [30]:
j_pix = np.floor(j_coord / [dim_info["x_res"], dim_info["y_res"], dim_info["z_res"]]).astype("uint16")
j_VARIABLE_labels = img[j_pix[:, 0], j_pix[:, 1], j_pix[:, 2]]
unique_labels = set(j_VARIABLE_labels)
print(f"Unique labels {len(unique_labels)}")
print(len([i for i in unique_labels if i in list(df_clean.original_labels)]))

Unique labels 1331
1128


In [31]:
tight_i = [np.where(matrix_i == i)[0][0] for i in np.unique(matrix_i)]
withme_i_VARIABLE = np.split(j_VARIABLE_labels, tight_i)[1:]

In [32]:
VARIABLE_per_i = [len(set(v).intersection(labels)) for v in withme_i_VARIABLE]

In [33]:
nombre = ESPECIMEN + "_" + "number" + "_" + str(d) + "myo.ply"  ## spl OR myo
file_out = os.path.join(folder_surface, nombre)
all_colors_median = np.zeros(shape=(mesh.vertices.shape[0], 4), dtype="float64")
colores_median = array2PLY.colorines(VARIABLE_per_i, cm.jet)
all_colors_median[np.unique(matrix_i)] = colores_median
mesh.visual.vertex_colors = all_colors_median
mesh.export(file_out)

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 5002\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nproperty uchar alpha\nelement face 10000\nproperty list uchar int vertex_indices\nend_header\n\r5\xafC\xe0\xb6UC\xc3\x15~C\x1f\xff\xd7\xff\xf4o\xf1C\xd6\xcegC\x07\xeb?C]\xff\x9a\xff\xdf/xC\x81#\x92CH\xe3\x8fC\xe7\xff\x0f\xff\xda\xf3\x9bC\xbf\xc1\x82C\xac\x11\x95C\xffY\x00\xff\xc13\xbcCgkgC|\xccFC\x1f\xff\xd7\xff\x98\xd4\xf3C\xd7VUC\xa2aEC\x00P\xff\xffp\x9a\x8aC\xde\xd3RC\xb8PeC\x00\x88\xff\xffZ\xd2\xe7C\xcf\x04bC\xb6\xeb7C\xff\xc1\x00\xffbf\tD\xba9\x97CT\x93\x8aC\x02\xe8\xf4\xff\x1a\xc0\x07D\x80S\x93C\nErC\x1f\xff\xd7\xffx\xa4\x01DEX8Ctb\x9dC\x00t\xff\xff\xecy\xacC\x84\x90hC\xc8\x7f\x92C\x1f\xff\xd7\xff\x07~\x97CumlC.v\x8cC\x00\xd4\xff\xff\xca\xa8\xd8C@\npC\xe4\x92AC\xc7\xff0\xff\x81N/D\xf4%\x96C\xb8vEC\x00\xd4\xff\xff\xb4\xbc\x1dD\xb5\xf8yC\x01\xfb;C\x0f\xf8\xe7\xff\x